In [ ]:
import csv
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import WebDriverException

chrome_driver_path = r"C:\Users\高奥成\AppData\Local\Programs\Python\Python311\chromedriver.exe"
chrome_binary_path = r"C:\Users\高奥成\AppData\Local\Google\Chrome\Application\chrome.exe"

# Chrome 옵션 생성
chrome_options = Options()
chrome_options.binary_location = chrome_binary_path
chrome_options.add_experimental_option("detach", True)
chrome_options.add_argument("--headless")  # 헤드리스 모드

# 모든 강의 정보를 저장할 리스트
all_course_info = []

# 페이지 1부터 57까지의 강의 정보 크롤링
for page_number in range(1, 58):
    try:
        # Chrome 웹드라이버 초기화
        service = Service(chrome_driver_path)
        driver = webdriver.Chrome(service=service, options=chrome_options)
        
        # 페이지 로드
        driver.get(f"https://www.inflearn.com/courses?page={page_number}")

        # 페이지 로드 완료 대기
        driver.implicitly_wait(10)  # 또는 WebDriverWait을 사용하여 페이지 요소가 로드될 때까지 대기

        # 페이지 소스 코드 가져오기
        page_source = driver.page_source

        # 페이지 소스 코드 파싱
        soup = BeautifulSoup(page_source, "html.parser")
        # 모든 강의 카드 항목 찾기
        course_cards = soup.find_all("article", class_="mantine-n8y7xk")

        # 각 강의 카드 항목 순회
        for course_card in course_cards:
            # 강의의 상세 링크 가져오기
            link_tag = course_card.find_parent("a")
            course_link = link_tag["href"] if link_tag else "N/A"

        # 강의 상세 정보를 포함하는 JSON 데이터 찾기
        script_tag = soup.find("script", id="__NEXT_DATA__", type="application/json")
        if script_tag:
            json_data = json.loads(script_tag.string)
            courses = json_data.get("props", {}).get("pageProps", {}).get("dehydratedState", {}).get("queries", [])[1].get("state", {}).get("data", {}).get("data", {}).get("items", [])

            # 각 강의를 순회하며 필요한 정보 추출
            for course_item in courses:
                course_data = course_item.get("course", {})
                instructor_data = course_item.get("instructor", {})
                list_price_data = course_item.get("listPrice", {})

                course_info = {
                    "course_id": course_data.get("id", "N/A"),
                    "course_link":course_link,
                    "slug": course_data.get("slug", "N/A"),
                    "thumbnail_url": course_data.get("thumbnailUrl", "N/A"),
                    "title": course_data.get("title", "N/A"),
                    "description": course_data.get("description", "N/A"),
                    "review_count": course_data.get("reviewCount", "N/A"),
                    "student_count": course_data.get("studentCount", "N/A"),
                    "like_count": course_data.get("likeCount", "N/A"),
                    "star_rating": course_data.get("star", "N/A"),
                    "updated_at": course_data.get("updatedAt", "N/A"),
                    "published_at": course_data.get("publishedAt", "N/A"),
                    "level": course_data.get("metadata", {}).get("level", "N/A"),
                    "skills": ", ".join(course_data.get("metadata", {}).get("skills", [])),
                    "parent_categories": ", ".join(course_data.get("metadata", {}).get("parentCategories", [])),
                    "child_categories": ", ".join(course_data.get("metadata", {}).get("childCategories", [])),
                    "instructor_id": instructor_data.get("id", "N/A"),
                    "instructor_name": instructor_data.get("name", "N/A"),
                    "pay_price": list_price_data.get("payPrice", "N/A"),
                    "regular_price": list_price_data.get("regularPrice", "N/A"),
                    "discount_rate": list_price_data.get("discountRate", "N/A"),
                    "discount_title": list_price_data.get("discountTitle", "N/A")
                }

                # 강의 정보를 강의 정보 리스트에 추가
                all_course_info.append(course_info)

        # 현재 WebDriver 인스턴스 종료
        driver.quit()
        service.stop()

        # 크롤링 완료 정보 출력
        print(f"{page_number} 페이지 크롤링 완료")

    except WebDriverException as e:
        print(f"{page_number} 페이지를 크롤링하는 동안 오류 발생")
        print("예외:", e)

# 강의 정보를 CSV 파일로 저장
csv_file_path = "courses_info.csv"
fieldnames = [
    "course_id", "course_link", "slug", "thumbnail_url", "title", "description", "review_count", 
    "student_count", "like_count", "star_rating", "updated_at", "published_at", 
    "level", "skills", "parent_categories", "child_categories", "instructor_id", 
    "instructor_name", "pay_price", "regular_price", "discount_rate", "discount_title"
]

with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    for course_info in all_course_info:
        writer.writerow(course_info)

print(f"강의 정보가 {csv_file_path}에 저장되었습니다")
